# 01 PUSH Type Import Strategy

## Use case
- Azure AI Search の サービス仕様ドキュメントをインプットにする。
  - https://learn.microsoft.com/ja-jp/azure/search/
- ドキュメントは OCR が必要。
- 開発者マニュアルは、構造化されたセクションとなっている。
- 各セクションは非常に詳細かつ専門性の高い技術解説が記載されており、ドキュメントサイズも大きい。
- ドキュメントには、テキスト、テーブル、図、グラフなどが含まれるが、ここでは、テキスト、テーブルデータのみを扱う。

## チャンキング設計
- Document Intelligence で、Markdown形式でテキストデータを抽出済み。
- 1つのドキュメントに大量のコンテキストが含まれており、ドキュメントサイズも大きいため、チャンキングを実施する。
- ドキュメントは、技術要素ごとに明確なセクションわけがされており、各セクションで見るとLLMが扱えないレベルのデータサイズではない。そのため、セクション単位でチャンキングする。
- 各チャンキングのContentはEmbeddingする。
- Overlapping は行わない。
- 広い意味のコンテキストを保持するために、上位2つのヘッダー（Markdown形式：#, ##）をメタデータとして保持する。例えば、検索インデックスに関する記載があった場合に、それが「キーワード検索」に属する情報なのか、「ベクトル検索」に属する情報なのかを判断するために保持する。

In [ ]:
! pip install python-dotenv langchain langchain-community langchain-openai langchainhub openai tiktoken azure-ai-documentintelligence azure-identity azure-search-documents==11.6.0b3

In [ ]:
"""
This code loads environment variables using the `dotenv` library and sets the necessary environment variables for Azure services.
The environment variables are loaded from the `.env` file in the same directory as this notebook.
"""
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
doc_intelligence_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")

In [ ]:
from langchain import hub
from langchain_openai import AzureChatOpenAI
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch

### Load markdown file

In [ ]:
def read_file(filename):
    with open(filename, 'r', encoding="utf-8") as f:
        content = f.read()
    return content

In [ ]:
markdown_content = read_file('../output/azure-search-concept.md')
print(markdown_content)

### Splitting with Langchain Text Splitter
https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/

We use `MarkdownHeaderTextSplitter` for markdown document.
  - https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/markdown_header_metadata/

In [ ]:
# Split the document into chunks base on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
# Include the headers in the splits.
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)

splits = text_splitter.split_text(markdown_content)

print("Length of splits: " + str(len(splits)))

In [ ]:
splits

# 03_Index-Design
Tips for Azure AI Search Index-Design

### PUSH型
- シンプルなパターンとして、ローカルのデータを検索インデックスにインポートさせる。
- ユーザのコンテキストを捉えるために、ベクトル検索を採用する。そのため、Embedding フィールドを構成する。
  - Embedding Model: `text-embedding-ada-002`を採用する。
- ドキュメントのヘッダー情報を`Metadata`フィールドに含める。
- ドキュメントのタイトルを`Title`フィールドに含める。

In [ ]:
from azure.search.documents.indexes.models import (
    ScoringProfile,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    TextWeights,
)

In [ ]:
aoai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2024-02-01",
)

vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")


fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name="ja.microsoft"
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(aoai_embeddings.embed_query("Text")),
        vector_search_profile_name="myHnswProfile",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name="ja.microsoft"
    ),
    # Additional field to store the title
    SearchableField(
        name="title",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name="ja.microsoft"
    ),
]

index_name: str = "rag-search-index-0619"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=aoai_embeddings.embed_query,
    fields=fields,
)

### 改善
不要な情報を検索インデックスに含めないようにする

In [ ]:
# Remove the specific chapter from the index
keywords = ["関連項目", "次のステップ"]
for split in splits:
    if any(keyword in value for value in split.metadata.values() for keyword in keywords):
        print(split.metadata)
        continue
    vector_store.add_texts(
		[split.page_content],
  		[
        	{"title": f"{os.path.basename('../data/azure-search-concept.pdf')}"}
        ]
	)

# 04_Query-Design
Tips for Azure AI Search Query-Design

### Simple Search
- ユーザのクエリのコンテキストをとらえたい、かつサービスに特化したワードがクエリに含まれる可能性が高いため、フルテキスト検索＋ベクトル検索を組み合わせたHybrid検索を採用する。
  - Hybrid検索のスコアはAzure AI Searchでは、Reciprocal Rank Fusion (RRF) が採用される。 
- クエリはユーザのクエリをそのまま検索インデックスのクエリに利用する。

In [ ]:
query="ベクトル検索時の設定要素について教えてください"
res_simple_query = vector_store.similarity_search(
    	query=query, k=3, search_type="hybrid"
)
res_simple_query

### Query Extensions
- ユーザのクエリのコンテキストをとらえたい、かつサービスに特化したワードがクエリに含まれる可能性が高いため、フルテキスト検索＋ベクトル検索を組み合わせたHybrid検索を採用する。
  - Hybrid検索のスコアはAzure AI Searchでは、Reciprocal Rank Fusion (RRF) が採用される。 
- また、ユーザクエリから検索クエリを新しく生成する。
  - クエリはユーザのクエリをスタンドアローンなクエリに変換する。
  - また、検索のカバレッジを大きくするために、類似した入力クエリを複数生成する。

In [ ]:
import re
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

system_message = """
# Your Task
- Given the following conversation history and the users next question,rephrase the question to be a stand alone question.
- You also need to extend the original question to generate 5 related queries. This is done to capture the broader context of the user's question.
- You must output json format. In other words, You must output array of questions that length is 5.

# Json format example:
{
	"questions": [
		"related question 1",
		"related question 2",
		"related question 3",
		"related question 4",
		"related question 5"
	]
}
"""

def correct_text_gpt(text):
    message_text = [
		{"role":"system","content": system_message},
		{"role":"user","content": text}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content



In [ ]:
query="ベクトル検索時の設定要素について教えてください"
expanded_query = correct_text_gpt(query)
parsed_data = json.loads(expanded_query)
parsed_data

In [ ]:
for question in parsed_data["questions"]:
	res_simple_query = vector_store.similarity_search(
		query=question, k=3, search_type="hybrid"
	)
	print(res_simple_query)
	print("\n")

### LangChain Query Expansion

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field


class ParaphrasedQuery(BaseModel):
    """You have performed query expansion to generate a paraphrasing of a question."""

    paraphrased_query: str = Field(
        ...,
        description="A unique paraphrasing of the original question.",
    )

In [ ]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \

Perform query expansion. If there are multiple common ways of phrasing a user question \
or common synonyms for key words in the question, make sure to return multiple versions \
of the query with the different phrasings.

If there are acronyms or words you are not familiar with, do not try to rephrase them.

Return at least 3 versions of the question."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
AzureChatOpenAI
llm = AzureChatOpenAI(
	openai_api_version="2024-02-01",
    azure_deployment="gpt-4o",
)
llm_with_tools = llm.bind_tools([ParaphrasedQuery])
query_analyzer = prompt | llm_with_tools | PydanticToolsParser(tools=[ParaphrasedQuery])

In [ ]:
expanded_query = query_analyzer.invoke(
    {
        "question": query,
    }
)
expanded_query

In [ ]:
for question in expanded_query:
	res_simple_query = vector_store.similarity_search(
		query=question.paraphrased_query, k=3, search_type="hybrid"
	)
	print(res_simple_query)
	print("\n")

### HyDE (Hypothetical Document Embeddings)

In [ ]:
import re
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)


def correct_text_gpt(text):
    hypothetical_gen_instruction = f"""Please write a passage to answer the question
	Question: {text}
	Passage:
	"""
    message_text = [
		{"role":"system","content": "You are an AI assistant."},
		{"role":"user","content": hypothetical_gen_instruction}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		# response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content



In [ ]:
query="ベクトル検索時の設定要素について教えてください"
hypothetical_answer = correct_text_gpt(query)
hypothetical_answer

In [ ]:
res_simple_query = vector_store.similarity_search(
		query=hypothetical_answer, k=3, search_type="hybrid"
	)
res_simple_query